## Imports

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [4]:
df = pd.read_csv('combined_df.csv')

In [5]:
df_relevant = df[['Type', 'Genres', 'Category', 'Normal_Rating_Count', 'Normal_Rating','Normal_Reviews', 'Normal_Size', 'Normal_Installs', 'Normal_Price', 'Normal_Revenue', 'Normal_popularity']]

In [ ]:
# Label Encoding for ordinal data
label_encoder = LabelEncoder()
dataset['Content Rating'] = label_encoder.fit_transform(dataset['Content Rating'])

# One-Hot Encoding for nominal data
onehot_encoder = OneHotEncoder(sparse=False)
encoded_genres = onehot_encoder.fit_transform(dataset[['Genres']])
encoded_genres_df = pd.DataFrame(encoded_genres, columns=onehot_encoder.get_feature_names_out(['Genres']))
dataset = pd.concat([dataset, encoded_genres_df], axis=1).drop(['Genres'], axis=1)

# Example using pandas get_dummies for nominal data
dataset = pd.get_dummies(dataset, columns=['Category'], drop_first=True)
